# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [39]:
!pip install "snowflake-connector-python[pandas]"


In [40]:
import pandas as pd
# Get user password
import getpass

pwd = getpass.getpass('Enter password')





Enter password··········


In [41]:
 #Create connection to snowflake
import snowflake.connector

conn = snowflake.connector.connect(
    user = 'lakanu',
    password = str(pwd),
    account = 'HMVDGNJ-PP86591'
)

### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [42]:
# Create a variable called sql and specify a query that it will store
sql = 'select * from COVID19_EPIDEMIOLOGICAL_DATA.PUBLIC.CDC_POLICY_MEASURES'


In [43]:
# Execute the Query using the python connector
res = conn.cursor().execute(sql).fetchall()

In [44]:
# Run that query, but this time use the read_sql method
# in the pandas data frame object
covid19_data = pd.read_sql(sql,conn)
covid19_data

<ipython-input-44-7da49d5f71d2>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  covid19_data = pd.read_sql(sql,conn)


,STATE_ID,COUNTY,FIPS_CODE,POLICY_LEVEL,DATE,POLICY_TYPE,START_STOP,COMMENTS,SOURCE,TOTAL_PHASE,ISO3166_1,ISO3166_2,LAST_UPDATE_DATE,LAST_REPORTED_FLAG
0,DE,None,NaN,state,2020-07-06,Phase 1,stop,Policy_Details: Governor Carney extended Delaw...,sip_submission_form: https://governor.delaware...,NaN,US,DE,2022-12-08 07:18:00,True
1,MS,Sunflower,28133.0,county,2020-07-20,Outdoor and Recreation,stop,Policy_Details: Executive Order by Governor Ta...,sip_submission_form: https://www.sos.ms.gov/co...,NaN,US,MS,2022-12-08 07:18:00,True
2,PR,None,NaN,state,2020-06-15,Non-Essential Businesses,stop,Policy_Details: Retail - 50% \nPersonal Care -...,sip_submission_form: https://www.estado.pr.gov...,NaN,US,PR,2022-12-08 07:18:00,True
3,MO,None,NaN,state,2020-06-15,Non-Essential Businesses,stop,Policy_Details: All industries: practice socia...,sip_submission_form: https://health.mo.gov/liv...,NaN,US,MO,2022-12-08 07:18:00,True
4,DE,None,NaN,state,2020-09-05,Phase 2,stop,Policy_Details: Governor Carney formally exten...,sip_submission_form: https://governor.delaware...,NaN,US,DE,2022-12-08 07:18:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4212,IL,Cass,17017.0,county,2021-01-15,Houses of Worship,start,Policy_Details: - All gatherings of 50 people ...,sip_submission_form: https://coronavirus.illin...,5.0,US,IL,2022-12-08 07:18:00,True
4213,NY,New York,36061.0,county,2020-06-08,Non-Essential Businesses,start,Policy_Details: New York City (All counties) t...,sip_submission_form: https://coronavirus.healt...,NaN,US,NY,2022-12-08 07:18:00,True
4214,TX,Morris,48343.0,county,2020-07-03,Mask Requirement,start,Policy_Details: County is approved to be exemp...,sip_submission_form: https://tdem.texas.gov/ga...,NaN,US,TX,2022-12-08 07:18:00,True
4215,ME,Piscataquis,23021.0,county,2020-05-18,Outdoor and Recreation,start,Policy_Details: Rural Reopening : wilderness ...,sip_submission_form: https://www.maine.gov/cov...,3.0,US,ME,2022-12-08 07:18:00,True


### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

In [45]:
# Convert DATE to datetime
covid19_data['DATE'] = pd.to_datetime(covid19_data['DATE'])

# Policy Implementation Over Time
# This will help us determine the frequency of policy implementations, in each state.
policy_over_time = covid19_data.groupby([pd.Grouper(key='DATE', freq='M'), 'POLICY_TYPE']).size().reset_index(name='counts')
print(policy_over_time)

          DATE                                POLICY_TYPE  counts
0   2020-01-31                Allow Audio Only Telehealth       2
1   2020-02-29                         State of Emergency       1
2   2020-03-31                Allow Audio Only Telehealth      39
3   2020-03-31  Allow Expand Medicaid Telehealth Coverage      48
4   2020-03-31                           Childcare (K-12)      16
..         ...                                        ...     ...
277 2021-06-30                                    Phase 5       1
278 2021-07-31                     Outdoor and Recreation       1
279 2021-07-31                                    Phase 5       1
280 2021-08-31                             Food and Drink       1
281 2021-08-31                           Mask Requirement       1

[282 rows x 3 columns]


### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

In [46]:
covid19_data.to_excel('covid19.xlsx', index=False)

---

## Submitting Your Work

Submit your work as usual